# GRPO Training project: teach an LLM to do additions, again

In this notebook, you'll find:
* A basic Transformer with basic tokenizer
* A basic dataset for additions
* A classical pre-trainer, minimizing cross entropy loss
* A Vanilla GRPO

You're not supposed to edit the existing code (you can if you want to...).
You should implement one (or more) of the following:
* GRPO with PPO (the `usual` one)
* RLOO
* ReMax
* DPO
* RAFT
* your own RLHF method!

### Task and Commment on the work

we ve implemented GRPO with PPO using the slides of the class.
You can see how in the `compute_loss_clip_KL`function

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

import random
import math
import re
import time

In [ ]:
num_digits = 3

dataset_size = 64_000
train_proportion = 0.9

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Step 1: Construct a tokenizer

In [ ]:
pad_token="[PAD]"
eos_token="[EOS]"

In [ ]:
class character_level_tokenizer:
    """
    character-level
    """
    def __init__(self):
        self.vocab = [str(x) for x in range(10)] + ["+", "="] + [pad_token, eos_token]
        self.token_to_id = {v : k for k, v in enumerate(self.vocab)}
        self.id_to_token = {k : v for k, v in enumerate(self.vocab)}
        self.ntokens = len(self.vocab)
        self.pattern = f"[^{re.escape(''.join(self.vocab))}]"

    def clean(self, text):
        """
        removes all characters not in the vocabulary
        """
        out = re.sub(self.pattern, "", text)
        return out

    def pre_tokenization(self, text):
        """
        character-level
        """
        return [c for c in text]

    def encode(self, text):
        text_list = self.pre_tokenization(self.clean(text))
        return [self.token_to_id[c] for c in text_list]

    def decode(self, token_list):
        return "".join([self.id_to_token[x] for x in token_list])

In [ ]:
tokenizer = character_level_tokenizer()
ntokens = tokenizer.ntokens
ntokens

14

In [ ]:
prompt = "12 + 42 ="
inputs = tokenizer.encode(prompt)
inputs, tokenizer.decode(inputs)

([1, 2, 10, 4, 2, 11], '12+42=')

## Step 2: Create a dataset for arithmetic operations

In [ ]:
def sample_datapoint(num_digits = 3):
    a_list = [random.randint(0, 9) for _ in range(num_digits)]
    b_list = [random.randint(0, 9) for _ in range(num_digits)]
    a_int = int("".join([str(x) for x in a_list]))
    b_int = int("".join([str(x) for x in b_list]))
    a_str = "".join([str(x) for x in a_list])
    b_str = "".join([str(x) for x in b_list])
    sum_int = a_int + b_int
    return (a_str + "+" + b_str + "=", str(sum_int))

sample_datapoint(3)

('224+830=', '1054')

In [ ]:
data = []
for _ in range(dataset_size):
    data.append(sample_datapoint(num_digits))
data[:4]

[('248+095=', '343'),
 ('827+001=', '828'),
 ('946+690=', '1636'),
 ('229+865=', '1094')]

In [ ]:
data_train = data[: int(train_proportion * dataset_size)]
data_test = data[int(train_proportion * dataset_size):]

len(data_train),len(data_test)

(57600, 6400)

## Step 3: Construct a model

In [ ]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class TransformerModel(nn.Transformer):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__(d_model=ninp,
                                               nhead=nhead,
                                               dim_feedforward=nhid,
                                               num_encoder_layers=nlayers)
        self.input_emb = nn.Embedding(ntoken, ninp)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        self.decoder = nn.Linear(ninp, ntoken)

        self.ninp = ninp
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        return torch.log(torch.tril(torch.ones(sz,sz)))

    def forward(self, src):
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        self.src_mask = mask

        src = self.input_emb(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output_enc = self.encoder(src, mask=self.src_mask)
        output_dec = self.decoder(output_enc)
        return F.log_softmax(output_dec, dim=-1), output_enc

In [ ]:
model = TransformerModel(ntoken = ntokens,
                         ninp = 128,
                         nhead = 16,
                         nhid = 64,
                         nlayers = 8)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


TransformerModel(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): Linear(in_features=128, out_features=14, bias=True)
  (input_emb): Embedding(14, 128)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
print("number of parameters: {}".format(sum([x.numel() for x in model.parameters()])))

number of parameters: 668942


### Useful functions

In [ ]:
def generate(model, prompts, new_tokens = 5, mode = "greedy", num_samples = 1, temperature = 0.8):
    input_tensor = torch.repeat_interleave(prompts, repeats = num_samples, dim = 1).to(device)
    # (prompt_length, batch_size * num_samples)
    for _ in range(new_tokens):
        output, _ = model(input_tensor) # (prompt_length, batch_size * num_samples, ntokens)
        logits = output[-1,:,:] # (batch_size * num_samples, ntokens)
        if mode == "greedy":
            tokens = torch.argmax(logits, -1).view((1,-1)) # (1, batch_size * num_samples)
        else: # mode == "sampling"
            logits /= temperature
            probs = torch.softmax(logits, dim=-1)
            tokens = torch.multinomial(probs, num_samples = 1).view((1,-1)) # (1, batch_size * num_samples)
        input_tensor = torch.cat((input_tensor, tokens), 0)
    return input_tensor

In [ ]:
model.eval()

prompt = "2+3="
prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
output = generate(model, prompt_tensor).view((1,-1))
output, tokenizer.decode(output[0].tolist())

(tensor([[ 2, 10,  3, 11, 12, 12, 12, 12, 12]], device='cuda:0'),
 '2+3=[PAD][PAD][PAD][PAD][PAD]')

In [ ]:
def pad(token_list, type_list = "prompts"):
    max_length = max([len(x) for x in token_list])
    out = []
    for x in token_list:
        if type_list == "prompts":
            out.append([tokenizer.token_to_id[pad_token]] * (max_length - len(x)) + x)
        if type_list == "answers":
            out.append(x + [tokenizer.token_to_id[eos_token]] + [tokenizer.token_to_id[pad_token]] * (max_length - len(x)))
    return out, max_length

In [ ]:
prompts = [tokenizer.encode("1+1="), tokenizer.encode("21+35=")]
answers = [tokenizer.encode("2"), tokenizer.encode("56")]
padded_prompts, _ = pad(prompts, "prompts")
padded_answers, _ = pad(answers, "answers")
padded_prompts, padded_answers
[tokenizer.decode(p) for p in padded_prompts], [tokenizer.decode(p) for p in padded_answers]

(['[PAD][PAD]1+1=', '21+35='], ['2[EOS][PAD]', '56[EOS]'])

In [ ]:
def get_batch(split, i, batch_size):
    data = data_train if split == 'train' else data_test

    prompts = [data[i][0] for i in range(i, i + batch_size)]
    encoded_prompts = [tokenizer.encode(prompt) for prompt in prompts]
    padded_prompts, prompt_length = pad(encoded_prompts, "prompts")

    answers = [data[i][1] for i in range(i, i + batch_size)]
    encoded_answers = [tokenizer.encode(answer) for answer in answers]
    padded_answers, answers_length = pad(encoded_answers, "answers")

    X = torch.stack([torch.tensor(x) for x in padded_prompts], 1)
    Y = torch.stack([torch.tensor(x) for x in padded_answers], 1)
    return X, Y, prompt_length, answers_length, prompts, answers

In [ ]:
X, Y, prompt_length, answers_length, prompts, answers = get_batch("train", 43, 16)
X.shape, Y.shape, prompt_length, answers_length, prompts[0], answers[0]

(torch.Size([8, 16]), torch.Size([5, 16]), 8, 4, '798+180=', '978')

## Step 4: Evaluate

In [ ]:
batch_size = 16

In [ ]:
def evaluate(batch_size = batch_size):
    # Turn on evaluation mode disables dropout.
    model.eval()
    correct = 0.
    with torch.no_grad():
        for batch, i in enumerate(range(0, len(data_test) - 1, batch_size)):
            prompts, target_answers, prompt_length, answers_length, _, _ = get_batch("test", i, batch_size)
            prompts = prompts.to(device) # (prompt_length, batch_size)
            target_answers = target_answers.to(device) # (answers_length + 1, batch_size)
            output = generate(model, prompts, answers_length + 1) # (prompt_length + answers_length + 1, batch_size)
            answers_tokens = output[prompt_length:, :] # (answers_length + 1, batch_size), contains tokens
            equality_test = answers_tokens == target_answers # (answers_length + 1, batch_size), contains boolean values
            correct += torch.all(equality_test, axis=0).float().sum()
        accuracy = correct / len(data_test)
    return accuracy.item()

In [ ]:
evaluate()

0.0

## Step 5: Train the model, classical approach

### Hyperparameters

In [ ]:
epochs = 5
batch_size = 16
learning_rate = 8e-4

reporting_per_epoch = 5
log_interval = len(data_train) // (reporting_per_epoch + 1)
assert(log_interval % batch_size == 0)

In [ ]:
def train():
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    best_test_accuracy = None
    test_accuracy = evaluate()
    print('-' * 89)
    print('| initialisation | test accuracy {:5.2f}'.format(test_accuracy))
    print('-' * 89)
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        total_loss = 0.
        start_time = time.time()
        for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):
            prompts, target_answers, prompt_length, answers_length, _, _ = get_batch("train", i, batch_size)
            prompts = prompts.to(device) # (prompt_length, batch_size)
            target_answers = target_answers.to(device) # (answers_length + 1, batch_size)
            input_tensor = torch.cat((prompts, target_answers), 0) # (prompt_length + answers_length + 1, batch_size)
            model.zero_grad()
            output, _ = model(input_tensor) # (prompt_length + answers_length + 1, batch_size, ntokens)
            output_answers = output[prompt_length-1:-1,:,:].reshape(-1, ntokens) # ((answers_length + 1) * batch_size, ntokens)
            target_answers = target_answers.view(-1)
            loss = F.cross_entropy(output_answers, target_answers)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if i % log_interval == 0 and batch > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - start_time
                print('| {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:5.2f} | perplexity {:8.2f}'.format(batch, len(data_train) // batch_size,
                                                                                                            elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
                total_loss = 0
                start_time = time.time()
        test_accuracy = evaluate()
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test accuracy {:5.2f}'.format(epoch, (time.time() - epoch_start_time), test_accuracy))
        print('-' * 89)
        # Save the model if the test accuracy is the best we've seen so far.
        if not best_test_accuracy or test_accuracy < best_test_accuracy:
            with open("arithmetic.pt", 'wb') as f:
                torch.save(model, f)
            best_test_accuracy = test_accuracy

In [ ]:
train()

-----------------------------------------------------------------------------------------
| initialisation | test accuracy  0.00
-----------------------------------------------------------------------------------------
|   600/ 3600 batches | ms/batch  1.08 | loss  0.09 | perplexity     1.09
|  1200/ 3600 batches | ms/batch  1.08 | loss  0.07 | perplexity     1.07
|  1800/ 3600 batches | ms/batch  1.03 | loss  0.07 | perplexity     1.07
|  2400/ 3600 batches | ms/batch  1.03 | loss  0.07 | perplexity     1.07
|  3000/ 3600 batches | ms/batch  1.04 | loss  0.07 | perplexity     1.07
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 69.64s | test accuracy  0.01
-----------------------------------------------------------------------------------------
|   600/ 3600 batches | ms/batch  1.09 | loss  0.07 | perplexity     1.07
|  1200/ 3600 batches | ms/batch  1.05 | loss  0.07 | perplexity     1.07
|  1800/ 3600 batches | ms/

In [ ]:
model.eval()

for i in range(20):
    prompt, answers = data_test[i]
    prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
    output = generate(model, prompt_tensor, len(answers) + 1).view((1,-1))
    print(tokenizer.decode(output.tolist()[0]) + "\t actual result: " + answers)

473+062=536[EOS]	 actual result: 535
952+626=1578[EOS]	 actual result: 1578
047+298=342[EOS]	 actual result: 345
684+038=722[EOS]	 actual result: 722
998+913=1912[EOS]	 actual result: 1911
631+917=1548[EOS]	 actual result: 1548
633+018=652[EOS]	 actual result: 651
955+024=978[EOS]	 actual result: 979
923+715=1638[EOS]	 actual result: 1638
258+551=806[EOS]	 actual result: 809
003+139=146[EOS]	 actual result: 142
647+546=1192[EOS]	 actual result: 1193
375+991=1369[EOS]	 actual result: 1366
095+163=258[EOS]	 actual result: 258
197+060=258[EOS]	 actual result: 257
461+411=872[EOS]	 actual result: 872
317+864=1182[EOS]	 actual result: 1181
294+007=302[EOS]	 actual result: 301
910+393=1302[EOS]	 actual result: 1303
161+206=369[EOS]	 actual result: 367


## Step 4 bis: Vanilla GRPO training

## Step 4 bis bis: PPO GRPO training

We'll use the PPO model. Actually the part we change is the computation loss

### Custom reward functions

In [ ]:
def accuracy_reward(output, answer):
    pattern = r"\[EOS\]"
    output = re.sub(pattern, "", output)
    pattern = r"(\[PAD\])*$"
    output = re.sub(pattern, "", output)
    return 1. if output == answer else 0.

accuracy_reward("123[EOS][PAD][PAD]", "123"), accuracy_reward("123", "124"),

(1.0, 0.0)

In [ ]:
def distance_accuracy_reward(output, answer):
    pattern = r"\[EOS\]"
    output = re.sub(pattern, "", output)
    pattern = r"(\[PAD\])*$"
    output = re.sub(pattern, "", output)
    int_output = int(output)
    int_answer = int(answer)
    return abs(int_output - int_answer) / max(int_output, int_answer)

distance_accuracy_reward("123[EOS]", "123"), distance_accuracy_reward("123[PAD]", "124"),

(0.0, 0.008064516129032258)

In [ ]:
def digit_accuracy_reward(output, answer):
    pattern = r"\[EOS\]"
    output = re.sub(pattern, "", output)
    pattern = r"(\[PAD\])*$"
    output = re.sub(pattern, "", output)
    return sum(c1 == c2 for (c1,c2) in zip(output, answer)) / max(len(output), len(answer))

digit_accuracy_reward("123[EOS][PAD][PAD]", "123"), digit_accuracy_reward("123[EOS]", "123"),

(1.0, 1.0)

In [ ]:
def reward_format(output):
    pattern = r"\d+\[EOS\](\[PAD\])*$"
    return 1. if bool(re.match(pattern, output)) else 0.

reward_format("123[EOS][PAD][PAD]"), reward_format("123[EOS]"), reward_format("123"),

(1.0, 1.0, 0.0)

### Hyperparameters

In [ ]:
epochs = 20
batch_size = 16
learning_rate = 1e-4
num_samples = 16
temperature = .8

reporting_per_epoch = 5
log_interval = len(data_train) // (reporting_per_epoch + 1)
assert(log_interval % batch_size == 0)

reward_fun = digit_accuracy_reward
reward_format = reward_format

In [ ]:
def compute_rewards(text_outputs, answers):
    repeated_answers = [answer for answer in answers for _ in range(num_samples)]
    rewards = torch.tensor(
        [0.2 * reward_format(output) + 0.8 * reward_fun(output, answer)
         for output, answer in zip(text_outputs, repeated_answers)],
        dtype=torch.float32,
        device=device
    )
    return rewards

In [ ]:
def calculate_grpo_advantages(rewards):
    # reshape rewards to group by prompt
    # compute mean and standard deviation for each prompt group
    mean_rewards = rewards.view(-1, num_samples).mean(dim=1)
    std_rewards = rewards.view(-1, num_samples).std(dim=1)
    # expand the means and stds to match the original flat rewards tensor shape
    mean_rewards = mean_rewards.repeat_interleave(num_samples, dim=0)
    std_rewards = std_rewards.repeat_interleave(num_samples, dim=0)
    # normalize rewards to get advantages
    advantages = (rewards - mean_rewards) / (std_rewards + 1e-5)
    return advantages

In [ ]:
def compute_log_probs(model, outputs, prompt_length):
    logits, _ = model(outputs)
    # logits.shape = (prompt_length + answers_length + 1, batch_size * num_samples, vocab_size)

    # we only need the log probabilities for the new tokens
    # this introduces a shift: the logits for a position are the predictions for the next token
    logits = logits[prompt_length-1:-1, :, :]
    # logits.shape = (answers_length + 1, batch_size * num_samples, vocab_size)

    # convert raw logits into log probabilities along the vocabulary axis
    log_probs = F.log_softmax(logits, dim=-1)
    # log_probs.shape = (answers_length + 1, batch_size * num_samples, vocab_size)
    return log_probs

In [ ]:
'''def compute_loss(advantages, log_probs, responses):
    # reshape responses from (answers_length + 1, batch_size * num_samples)
    # to (answers_length + 1, batch_size * num_samples, 1) for gathering
    responses = responses.unsqueeze(-1)
    # log_probs.shape = (answers_length + 1, batch_size * num_samples, vocab_size)
    # responses.shape = (answers_length + 1, batch_size * num_samples)
    # gather the log probability for each token in responses
    selected_log_probs = log_probs.gather(dim=-1, index=responses)
    # remove the extra last dimension to get back to shape (answers_length + 1, batch_size * num_samples).
    selected_log_probs = selected_log_probs.squeeze(-1)

    # normalize
    selected_log_probs = (selected_log_probs - selected_log_probs.mean(-1, keepdim=True)) / (selected_log_probs.std(-1, keepdim=True) + 1e-5)

    # advantages.shape = (batch_size * num_samples)
    # we use the same advantages for all tokens in the response
    loss = -(advantages.unsqueeze(dim=0) * selected_log_probs).mean()
    return loss'''

'def compute_loss(advantages, log_probs, responses):\n    # reshape responses from (answers_length + 1, batch_size * num_samples)\n    # to (answers_length + 1, batch_size * num_samples, 1) for gathering\n    responses = responses.unsqueeze(-1)\n    # log_probs.shape = (answers_length + 1, batch_size * num_samples, vocab_size)\n    # responses.shape = (answers_length + 1, batch_size * num_samples)\n    # gather the log probability for each token in responses\n    selected_log_probs = log_probs.gather(dim=-1, index=responses)\n    # remove the extra last dimension to get back to shape (answers_length + 1, batch_size * num_samples).\n    selected_log_probs = selected_log_probs.squeeze(-1)\n\n    # normalize\n    selected_log_probs = (selected_log_probs - selected_log_probs.mean(-1, keepdim=True)) / (selected_log_probs.std(-1, keepdim=True) + 1e-5)\n\n    # advantages.shape = (batch_size * num_samples)\n    # we use the same advantages for all tokens in the response\n    loss = -(advantag

In [ ]:
def compute_loss_clip_KL(advantages, log_probs, old_log_probs, responses, epsilon=0.2, beta=0.03):
    responses = responses.unsqueeze(-1)

    # Gather log probabilities for selected responses
    selected_log_probs = log_probs.gather(dim=-1, index=responses).squeeze(-1)
    old_selected_log_probs = old_log_probs.gather(dim=-1, index=responses).squeeze(-1)

    # Compute probability ratio
    ratio = torch.exp(selected_log_probs - old_selected_log_probs)

    # Compute clipped surrogate objective
    clipped_ratio = torch.clamp(ratio, 1 - epsilon, 1 + epsilon)

    # Compute PPO loss
    advantages_unsqueezed = advantages.unsqueeze(dim=0)  # Unused advantage computation reused
    ppo_loss = -torch.min(ratio * advantages_unsqueezed, clipped_ratio * advantages_unsqueezed).mean()

    # Compute KL divergence (Regularization Term)
    # Using log-probabilities for KL divergence
    kl_div = F.kl_div(selected_log_probs.log(), old_selected_log_probs.exp(), reduction='batchmean')

    # Total loss: PPO loss + KL penalty
    loss = ppo_loss + beta * kl_div

    return loss


In [ ]:
def compute_value(model, prompts):
    # Use the model's output to estimate the value function
    # You can use the last hidden state or apply a specific layer to output a scalar value
    value = model(prompts)  # Assuming the model outputs logits or embeddings
    value = value.mean(dim=-1)  # Example: Average over the sequence length to get a scalar value
    return value


In [ ]:
def compute_value(model, prompts):
    # Assuming 'model' has a separate value head for the value function
    value = model.value_head(prompts)  # Model should output values for each state
    return value


In [ ]:
'''

def train_ppo(verbose=False):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    best_test_accuracy = None
    test_accuracy = evaluate()
    print('-' * 89)
    print('| initialization | test accuracy {:5.2f}'.format(test_accuracy))
    print('-' * 89)

    # switch eval for train model (enables dropout)
    model.train()

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        start_time = time.time()
        old_log_probs = []  # Store the old log probs
        values = []  # Store values from the value network
        advantages = []  # Store advantages
        rewards = []  # Store rewards

        for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):

            # get a batch of prompts and answers
            prompts, _, prompt_length, answers_length, questions, answers = get_batch("train", i, batch_size)
            prompts = prompts.to(device)  # (prompt_length, batch_size)

            # generate samples for each prompt
            outputs = generate(model,
                               prompts,
                               new_tokens=answers_length + 1,
                               mode="sampling",
                               num_samples=num_samples,
                               temperature=temperature)
            # outputs.shape = (prompt_length + answers_length + 1, batch_size * num_samples)
            text_outputs = [tokenizer.decode(outputs[prompt_length:, i].tolist())
                            for i in range(outputs.size(1))]

            # compute rewards
            reward = compute_rewards(text_outputs, answers)
            rewards.append(reward)

            # Compute advantages (here using GAE or some form of advantage calculation)
            advantage = calculate_grpo_advantages(reward)
            advantages.append(advantage)

            # Compute old log probabilities (for PPO ratio)
            log_probs = compute_log_probs(model, outputs, prompt_length)
            old_log_probs.append(log_probs)

            # Compute values from the value network (if applicable)
            value = compute_value(model, prompts)  # Assuming model has a value network
            values.append(value)


        # Convert rewards, advantages, old log_probs to tensors
        rewards = torch.tensor(rewards).to(device)
        advantages = torch.tensor(advantages).to(device)
        old_log_probs = torch.tensor(old_log_probs).to(device)
        values = torch.tensor(values).to(device)

        # Normalize advantages if necessary
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # PPO optimization loop
        for _ in range(ppo_epochs):  # PPO typically performs multiple epochs of optimization
            for batch in range(len(rewards)):

                # Calculate the ratio (new_prob / old_prob)
                ratio = torch.exp(log_probs - old_log_probs)

                # Calculate surrogate objective with clipping
                clipped_ratio = torch.clamp(ratio, 1 - epsilon, 1 + epsilon)
                surrogate_loss = torch.min(ratio * advantages[batch], clipped_ratio * advantages[batch])

                # Value loss (typically MSE loss)
                value_loss = torch.nn.functional.mse_loss(values[batch], rewards[batch])

                # Entropy bonus to encourage exploration (optional)
                entropy_loss = -torch.mean(torch.log(ratio))  # Can also be based on entropy from the model's predictions

                # Total loss (can include weightings for each term)
                loss = -surrogate_loss.meaprinn() + value_loss.mean() - entropy_loss.mean()

                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        test_accuracy = evaluate()
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test accuracy {:5.2f}'.format(epoch, (time.time() - epoch_start_time), test_accuracy))
        print('-' * 89)

        # Save the model if the test accuracy is the best we've seen so far.
        if not best_test_accuracy or test_accuracy < best_test_accuracy:
            with open("arithmetic_ppo.pt", 'wb') as f:
                torch.save(model, f)
            best_test_accuracy = test_accuracy
'''

'\n\ndef train_ppo(verbose=False):\n    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)\n    best_test_accuracy = None\n    test_accuracy = evaluate()\n    print(\'-\' * 89)\n    print(\'| initialization | test accuracy {:5.2f}\'.format(test_accuracy))\n    print(\'-\' * 89)\n\n    # switch eval for train model (enables dropout)\n    model.train()\n\n    for epoch in range(1, epochs + 1):\n        epoch_start_time = time.time()\n        start_time = time.time()\n        old_log_probs = []  # Store the old log probs\n        values = []  # Store values from the value network\n        advantages = []  # Store advantages\n        rewards = []  # Store rewards\n\n        for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):\n\n            # get a batch of prompts and answers\n            prompts, _, prompt_length, answers_length, questions, answers = get_batch("train", i, batch_size)\n            prompts = prompts.to(device)  # (prompt_length, batch_size

In [ ]:
import torch
import time

def train_ppo_GRPO(verbose=False):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    best_test_accuracy = None
    test_accuracy = evaluate()
    print('-' * 89)
    print('| initialisation | test accuracy {:5.2f}'.format(test_accuracy))
    print('-' * 89)

    # Switch to train mode (enables dropout, etc.)
    model.train()

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        start_time = time.time()

        for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):

            # Get a batch of prompts and answers
            prompts, _, prompt_length, answers_length, questions, answers = get_batch("train", i, batch_size)
            prompts = prompts.to(device)  # (prompt_length, batch_size)

            # Generate samples for each prompt
            outputs = generate(model,
                               prompts,
                               new_tokens=answers_length + 1,
                               mode="sampling",
                               num_samples=num_samples,
                               temperature=temperature)

            # outputs.shape = (prompt_length + answers_length + 1, batch_size * num_samples)
            text_outputs = [tokenizer.decode(outputs[prompt_length:, i].tolist())
                            for i in range(outputs.size(1))]

            # Compute rewards
            rewards = compute_rewards(text_outputs, answers)

            # Compute advantages using GRPO method
            advantages = calculate_grpo_advantages(rewards)

            # Compute log probabilities for the generated outputs
            log_probs = compute_log_probs(model, outputs, prompt_length)

            # Compute old log probabilities to compute the PPO loss
            old_log_probs = log_probs.detach()  # Detach old log probs for comparison

            # Compute loss
            responses = outputs[prompt_length:, :]
            loss = compute_loss_clip_KL(advantages, log_probs, old_log_probs, responses)

            # Backpropagate the loss and optimize
            optimizer.zero_grad()  # Ensure gradients are zeroed out at the start of the backward pass
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Print statistics periodically
            if i % log_interval == 0 and batch > 0:
                elapsed = time.time() - start_time
                print('| {:5d}/{:5d} batches | ms/batch {:5.2f}'.format(batch, len(data_train) // batch_size, elapsed))
                if verbose:
                    print("\nquestion:", questions[0],
                          "\nanswer:", answers[0],
                          "\noutput:", text_outputs[:num_samples],
                          "\nreward:", rewards[:num_samples],
                          "\nadvantage:", advantages[:num_samples], "\n")

                start_time = time.time()

        # Evaluate the model at the end of each epoch
        test_accuracy = evaluate()
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test accuracy {:5.2f}'.format(epoch, (time.time() - epoch_start_time), test_accuracy))
        print('-' * 89)

        # Save the model if the test accuracy is the best we've seen so far
        if best_test_accuracy is None or test_accuracy > best_test_accuracy:
            with open("ppo_GRPO.pt", 'wb') as f:
                torch.save(model, f)
            best_test_accuracy = test_accuracy


In [ ]:
'''def train_ppo_GRPO(verbose = False):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    best_test_accuracy = None
    test_accuracy = evaluate()
    print('-' * 89)
    print('| initialisation | test accuracy {:5.2f}'.format(test_accuracy))
    print('-' * 89)

    # switch eval for train model (enables dropout)
    model.train()

    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        start_time = time.time()
        for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):

            # get a batch of prompts and answers
            prompts, _, prompt_length, answers_length, questions, answers = get_batch("train", i, batch_size)
            prompts = prompts.to(device) # (prompt_length, batch_size)

            # generate samples for each prompt
            outputs = generate(model,
                               prompts,
                               new_tokens = answers_length + 1,
                               mode = "sampling",
                               num_samples = num_samples,
                               temperature = temperature)
            # outputs.shape = (prompt_length + answers_length + 1, batch_size * num_samples)
            text_outputs = [tokenizer.decode(outputs[prompt_length:, i].tolist())
                            for i in range(outputs.size(1))]

            # compute rewards
            rewards = compute_rewards(text_outputs, answers)

            # compute advantages
            advantages = calculate_grpo_advantages(rewards)

            # compute log probabilities
            log_probs = compute_log_probs(model, outputs, prompt_length)

            # compute loss
            responses = outputs[prompt_length:, :]
            loss = compute_loss_clip_KL(advantages, log_probs, old_log_probs, responses)

            # optimize
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i % log_interval == 0 and batch > 0:
                elapsed = time.time() - start_time
                print('| {:5d}/{:5d} batches | ms/batch {:5.2f}'.format(batch, len(data_train) // batch_size, elapsed))
                if verbose:
                    print("\nquestion:", questions[0],
                      "\nanswer", answers[0],
                      "\noutput:", text_outputs[:num_samples],
                      "\nreward:", rewards[:num_samples],
                      "\nadvantage:", advantages[:num_samples], "\n")

                start_time = time.time()
        test_accuracy = evaluate()
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test accuracy {:5.2f}'.format(epoch, (time.time() - epoch_start_time), test_accuracy))
        print('-' * 89)
        # Save the model if the test accuracy is the best we've seen so far.
        if not best_test_accuracy or test_accuracy < best_test_accuracy:
            with open("ppo_GRPO.pt", 'wb') as f:
                torch.save(model, f)
            best_test_accuracy = test_accuracy '''


In [ ]:
train_ppo_GRPO(verbose = False)

-----------------------------------------------------------------------------------------
| initialisation | test accuracy  0.19
-----------------------------------------------------------------------------------------
|   600/ 3600 batches | ms/batch 54.22
|  1200/ 3600 batches | ms/batch 53.78
|  1800/ 3600 batches | ms/batch 53.74
|  2400/ 3600 batches | ms/batch 53.61
|  3000/ 3600 batches | ms/batch 53.71
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 331.81s | test accuracy  0.00
-----------------------------------------------------------------------------------------
|   600/ 3600 batches | ms/batch 51.09
|  1200/ 3600 batches | ms/batch 50.60
|  1800/ 3600 batches | ms/batch 50.80
|  2400/ 3600 batches | ms/batch 50.73
|  3000/ 3600 batches | ms/batch 50.71
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 313.99s | test accuracy  0.10
-------

In [ ]:
model.eval()

for i in range(20):
    prompt, answers = data_test[i]
    prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
    output = generate(model, prompt_tensor, len(answers) + 1).view((1,-1))
    print(tokenizer.decode(output.tolist()[0]) + "\t actual result: " + answers)

473+062=535[EOS]	 actual result: 535
952+626=1578[EOS]	 actual result: 1578
047+298=345[EOS]	 actual result: 345
684+038=722[EOS]	 actual result: 722
998+913=1910[EOS]	 actual result: 1911
631+917=1548[EOS]	 actual result: 1548
633+018=651[EOS]	 actual result: 651
955+024=979[EOS]	 actual result: 979
923+715=1638[EOS]	 actual result: 1638
258+551=809[EOS]	 actual result: 809
003+139=142[EOS]	 actual result: 142
647+546=1193[EOS]	 actual result: 1193
375+991=1366[EOS]	 actual result: 1366
095+163=258[EOS]	 actual result: 258
197+060=257[EOS]	 actual result: 257
461+411=872[EOS]	 actual result: 872
317+864=1181[EOS]	 actual result: 1181
294+007=301[EOS]	 actual result: 301
910+393=1303[EOS]	 actual result: 1303
161+206=367[EOS]	 actual result: 367
